In [1]:
import time
import json
import random
import requests
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
def getDataFromPage(url):
    request = requests.get(url)
    data =  None

    if request.status_code == 200:
        soup = BeautifulSoup(request.text, "html.parser")

        data = soup.find_all('script')
        split_data = data[7].text.split("window.__INITIAL_STATE__=")
        data = split_data[1]
    return data

In [3]:
def getPageCount(data):
    result = data["resultList"]["search"]["fullSearch"]["result"]
    page_count = result["pageCount"]

    return page_count

In [4]:
def createDataFrame(url):
    list_rent:list = []
    data = getDataFromPage(url)

    if data:
        json_data = json.loads(data)
        num_pages = getPageCount(json_data)

        for page in tqdm(range(1, num_pages + 1)):
            url_page = f"https://www.immoscout24.ch/de/immobilien/mieten/ort-basel?pn={page}"

            try:
              scraping_data = getDataFromPage(url_page)
              json_scraping_data = json.loads(scraping_data)
              scraping_result = json_scraping_data["resultList"]["search"]["fullSearch"]["result"]
              listings = scraping_result["listings"]
              list_rent.append(listings)
              print(f"Count: {page}")
              time.sleep(random.randint(1, 4))
            except:
              continue

    return list_rent

In [5]:
def createDataset(data):
  dataset:list = []

  try:

    for page in enumerate(data):
      for listing in enumerate(page[1]):

        street = listing[1]["listing"]["address"].get("street") or None,
        located = listing[1]["listing"]["address"].get("postalCode") or None,
        characteristics = listing[1]["listing"].get('characteristics') or None

        if characteristics is not None:
          dataset.append(
              {
                  "id": listing[1].get("id") or None,
                  "categories": listing[1]["listing"].get("categories") or None,
                  "price (CHF)": listing[1]["listing"]["prices"]["rent"].get("gross") or None,
                  "latitude": listing[1]["listing"]["address"]["geoCoordinates"].get("latitude") or None,
                  "longitude": listing[1]["listing"]["address"]["geoCoordinates"].get("longitude") or None,
                  "city": listing[1]["listing"]["address"].get("locality") or None,
                  "living_space (m2)": characteristics.get("livingSpace") or None,
                  "num_rooms": characteristics.get("numberOfRooms") or None,
                  "floor": characteristics.get("floor") or None,
                  "num_floors": characteristics.get("numberOfFloors") or None,
                  "year_built": characteristics.get("yearBuilt") or None,
                  "is_new_building": characteristics.get("isNewBuilding") or None,
                  "is_old_building": characteristics.get("isOldBuilding") or None,
                  "year_last_renovated": characteristics.get("yearLastRenovated") or None,
                  "street": street[0] if street is not None else None,
                  "postal_code": located[0] if located is not None else None,
              }
          )
  except NameError as e:
    print(e)
  return dataset

In [6]:
# URL BASE
url = "https://www.immoscout24.ch/de/immobilien/mieten/ort-basel?nrs=100"
# DATA
data = createDataFrame(url)

  0%|          | 0/48 [00:00<?, ?it/s]

Count: 1


  2%|▏         | 1/48 [00:02<02:12,  2.82s/it]

Count: 2


  4%|▍         | 2/48 [00:06<02:34,  3.37s/it]

...

 42%|████▏     | 20/48 [01:12<01:37,  3.47s/it]

Count: 21


 44%|████▍     | 21/48 [01:14<01:20,  2.97s/it]

Count: 22


100%|██████████| 48/48 [01:17<00:00,  1.61s/it]


In [7]:
dataset = createDataset(data)

In [8]:
df = pd.DataFrame(dataset)
df.to_csv("rents_basel.csv", index=False)